### IMPORT REQUIRED MODULES

In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn

In [20]:
import pandas as pd
import os
from pathlib import Path
import numpy as np
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import GroupKFold
from datetime import datetime, date

### DATA CLEANING

#### MERGE ALL DATA

In [15]:
def merge_csv_files(directory_path):
    # Get all CSV files in the directory
    csv_files = list(Path(directory_path).glob('*.csv'))
    
    # Initialize list to store dataframes
    dfs = []
    
    # Read each CSV file and add filename as a column
    for file in csv_files:
        df = pd.read_csv(file, header=0)
        filename = file.stem  # Get filename without extension
        df['Event Code'] = filename
        dfs.append(df)
    
    # Concatenate all dataframes
    merged_df = pd.concat(dfs, ignore_index=True)

    # Filter out "Booker not confirmed" rows
    merged_df = merged_df[merged_df['Attendee Status'] != 'Booker not attending']
    
    # Export merged dataframe
    output_path = os.path.join("/workspaces/collaborative_app_dev/Data/Cleaned Data", 'merged_data.csv')
    merged_df.to_csv(output_path, index=False)
    
    return output_path

In [16]:
# Usage
directory = "/workspaces/collaborative_app_dev/Data/Raw Data"
output_file = merge_csv_files(directory)
df = pd.read_csv("/workspaces/collaborative_app_dev/Data/Cleaned Data/merged_data.csv")
print(len(df))

7417


#### GROUP TICKETS BY DATE CREATED BASED ON EVENT

In [4]:
def create_event_dates_dict():
    # Dictionary mapping event codes to their dates and target audience
    return {
        'D19': {'date': '2019-11-19', 'audience': 'IT Managers'},
        'D21': {'date': '2021-12-09', 'audience': 'IT Managers'},
        'D24': {'date': '2024-10-03', 'audience': 'IT Managers'},
        'GP21': {'date': '2021-04-22', 'audience': 'Property Managers'},
        'GP24': {'date': '2024-09-11', 'audience': 'Property Managers'},
        'MSE21': {'date': '2021-03-24', 'audience': 'Education property managers'},
        'NP21': {'date': '2021-11-09', 'audience': 'Property Managers'},
        'NP24': {'date': '2024-11-06', 'audience': 'Property Managers'},
        'SRM22': {'date': '2022-06-15', 'audience': 'Education Managers'},
        'SRM23': {'date': '2023-06-08', 'audience': 'Education Managers'}
    }

In [18]:
def analyze_registrations(merged_csv_path):
    # Read the merged CSV
    df = pd.read_csv(merged_csv_path)
    
    # Convert date_created to datetime if it's not already
    df['Created Date'] = pd.to_datetime(df['Created Date'])
    
    # Group by date and event (Event Code) and count registrations
    registration_counts = df.groupby([
        df['Created Date'].dt.date,
        'Event Code'
    ]).size().reset_index(name='registration_count')
    
    # Sort by date and event
    registration_counts = registration_counts.sort_values(['Created Date', 'Event Code'])

#-----------------------------------------------------------------------------------
    # Calculate cumulative registrations for each event separately
    registration_counts['cumulative_registrations'] = registration_counts.groupby('Event Code')['registration_count'].cumsum()
    

    # Add event dates and target audience
    event_dates = create_event_dates_dict()
    
    # Add event date and target audience columns
    registration_counts['Event date'] = registration_counts['Event Code'].map(
        {k: pd.to_datetime(v['date']) for k, v in event_dates.items()}
    )

    # Remove registrations after event date
    registration_counts = registration_counts[registration_counts['Created Date'] <= registration_counts['Event date']]
    
    registration_counts['Target audience'] = registration_counts['Event Code'].map(
        {k: v['audience'] for k, v in event_dates.items()}
    )

    # Calculate days until event
    registration_counts['Days until event'] = (
        registration_counts['Event date'] - pd.to_datetime(registration_counts['Created Date'])
    ).dt.days


    # Sort by date and event for final output
    registration_counts = registration_counts.sort_values(['Created Date', 'Event Code'])
#--------------------------------------------------------------------------------------


    # Detect promotional spikes
    mean_daily = registration_counts['registration_count'].mean()
    std_daily = registration_counts['registration_count'].std()
    registration_counts['promotional_spike'] = (registration_counts['registration_count'] > 
                                        (mean_daily + 2 * std_daily)).astype(int)
    
    # ---------------------------------------------------------------------------------------

    
    # Export the analysis
    analysis_path = os.path.join(os.path.dirname("/workspaces/collaborative_app_dev/Data/Cleaned Data/"), 'complete_registration_analysis.csv')
    registration_counts.to_csv(analysis_path, index=False)
    
    return analysis_path

In [19]:
analysis_path = analyze_registrations("/workspaces/collaborative_app_dev/Data/Cleaned Data/merged_data.csv")

df = pd.read_csv(analysis_path)
df.head(10)

/tmp/ipykernel_6827/1299981618.py:6: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['Created Date'] = pd.to_datetime(df['Created Date'])


,Created Date,Event Code,registration_count,cumulative_registrations,Event date,Target audience,Days until event,promotional_spike
0,2019-07-16,D19,125,125,2019-11-19,IT Managers,126,1
1,2019-07-30,D19,118,243,2019-11-19,IT Managers,112,1
2,2019-07-31,D19,19,262,2019-11-19,IT Managers,111,0
3,2019-08-01,D19,2,264,2019-11-19,IT Managers,110,0
4,2019-08-02,D19,1,265,2019-11-19,IT Managers,109,0
5,2019-08-05,D19,5,270,2019-11-19,IT Managers,106,0
6,2019-08-06,D19,2,272,2019-11-19,IT Managers,105,0
7,2019-08-07,D19,3,275,2019-11-19,IT Managers,104,0
8,2019-08-08,D19,5,280,2019-11-19,IT Managers,103,0
9,2019-08-09,D19,19,299,2019-11-19,IT Managers,102,0


### FEATURE ENGINEERING AND EXTRACTION

In [7]:
data = pd.read_csv("/workspaces/collaborative_app_dev/Data/Cleaned Data/complete_registration_analysis.csv")

In [8]:
data.head()
#len(data)

,Created Date,Event Code,registration_count,cumulative_registrations,Event date,Target audience,Days until event,promotional_spike
0,2019-07-16,D19,125,125,2019-11-19,IT Managers,126,1
1,2019-07-30,D19,118,243,2019-11-19,IT Managers,112,1
2,2019-07-31,D19,19,262,2019-11-19,IT Managers,111,0
3,2019-08-01,D19,2,264,2019-11-19,IT Managers,110,0
4,2019-08-02,D19,1,265,2019-11-19,IT Managers,109,0


In [9]:
current_date = date.today()
print(type(pd.to_datetime(current_date)))


diff = (pd.to_datetime(current_date) - pd.to_datetime(data['Created Date'][34])).days
print(diff)

days = (pd.to_datetime(data['Event date'][1]) - pd.to_datetime(current_date)).days
print(days)

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
1964
-1911


In [10]:
def extract_train_features(df):
    """Extract features with enhanced pattern detection"""
    features = []
    targets = []

    for event_code in df['Event Code'].unique():
        event_df = df[df['Event Code'] == event_code].copy()
        
        registrations_at_point = event_df
        
        if len(registrations_at_point) == 0:
            continue

        # Enhanced feature extraction
        recent_registrations = registrations_at_point.tail(7)['registration_count'].mean()
        early_registrations = registrations_at_point.head(7)['registration_count'].mean()

        #--------------------------------------------------------------------------------------------
        # Calculate days since last spike
        spike_mask = registrations_at_point['promotional_spike'] == 1
        if any(spike_mask):
            # Get the most recent spike's days until event
            last_spike_days = registrations_at_point[spike_mask]['Days until event'].max()
            # Calculate the difference between current point and last spike
            days_since_spike = registrations_at_point['Days until event'].max() - last_spike_days
        else:
            # If no spikes, use 0 to indicate no prior spikes have occurred
            days_since_spike = 0
        #---------------------------------------------------------------------------------------------


        feature_dict = {
            'current_registrations': registrations_at_point['cumulative_registrations'].max(),
            'avg_daily_rate': registrations_at_point['registration_count'].mean(),
            'recent_velocity': recent_registrations,
            'early_velocity': early_registrations,
            'registration_acceleration': (recent_registrations - early_registrations) / 7,
            'days_active': len(registrations_at_point),
            'peak_daily_registrations': registrations_at_point['registration_count'].max(),
            'registration_volatility': registrations_at_point['registration_count'].std(),
            #-----------------------------------------------------------------------------
            'spike_count': registrations_at_point['promotional_spike'].sum(),
            'days_since_last_spike': days_since_spike,
            'event_code': registrations_at_point['Event Code'].unique()[0],
            'target_audience': registrations_at_point['Target audience'].unique()[0],
            'registration_start_date': registrations_at_point['Created Date'].unique()[0],
            'event_date': registrations_at_point['Event date'].unique()[0]
            }


        features.append(feature_dict)
        targets.append(event_df['cumulative_registrations'].max())

    return pd.DataFrame(features), np.array(targets)


In [11]:
features, target = extract_train_features(df=data)
features.head(10)

,current_registrations,avg_daily_rate,recent_velocity,early_velocity,registration_acceleration,days_active,peak_daily_registrations,registration_volatility,spike_count,days_since_last_spike,event_code,target_audience,registration_start_date,event_date
0,1172,16.507042,49.714286,38.857143,1.551020,71,125,24.387862,8,0,D19,IT Managers,2019-07-16,2019-11-19
1,798,9.614458,15.285714,14.714286,0.081633,83,72,12.679762,2,258,GP21,Property Managers,2020-02-06,2021-04-22
2,395,4.593023,7.714286,11.571429,-0.551020,86,53,7.286374,1,8,NP21,Property Managers,2020-09-02,2021-11-09
3,669,7.779070,25.428571,1.142857,3.469388,86,68,10.292660,1,399,D21,IT Managers,2020-11-02,2021-12-09
4,1593,28.963636,23.714286,19.857143,0.551020,55,135,32.753831,11,0,MSE21,Education property managers,2021-01-19,2021-03-24
5,940,12.876712,14.571429,22.857143,-1.183673,73,157,23.122827,5,0,SRM22,Education Managers,2022-03-16,2022-06-15
6,699,9.708333,8.142857,18.714286,-1.510204,72,92,15.805541,3,31,SRM23,Education Managers,2023-01-30,2023-06-08
7,391,6.015385,13.285714,3.285714,1.428571,65,22,5.260384,0,0,D24,IT Managers,2024-01-05,2024-10-03
8,317,4.731343,4.857143,11.285714,-0.918367,67,36,6.236700,0,0,GP24,Property Managers,2024-04-15,2024-09-11
9,311,4.936508,5.285714,5.571429,-0.040816,63,18,4.219284,0,0,NP24,Property Managers,2024-04-25,2024-11-06


In [14]:
print(target)

[1172  798  395  669 1593  940  699  391  317  311]


### TRAINING

### PREDICTION

#### EXTRACT FEATURES FOR PREDICTION, NEEDS DATE AS INPUT

In [43]:
def extract_features(df, event_date, date_at_point):
    """Extract features with enhanced pattern detection"""

    #days = (pd.to_datetime(df['Event date'][0]) - pd.to_datetime(date_at_point)).days
    days = (pd.to_datetime(event_date) - pd.to_datetime(date_at_point)).days
    features = []

    if days < 0:
        print("Enter a date on or before the Event date")

    else:
        for event_code in df['Event Code'].unique():
            event_df = df[df['Event Code'] == event_code].copy()
            
            if event_df['Days until event'].max() < days:
                continue
            
            registrations_at_point = event_df[event_df['Days until event'] >= days]
            
            if len(registrations_at_point) == 0:
                continue

            # Enhanced feature extraction
            recent_registrations = registrations_at_point.tail(7)['registration_count'].mean()
            early_registrations = registrations_at_point.head(7)['registration_count'].mean()

            #--------------------------------------------------------------------------------------------
            # Calculate days since last spike
            spike_mask = registrations_at_point['promotional_spike'] == 1
            if any(spike_mask):
                # Get the most recent spike's days until event
                last_spike_days = registrations_at_point[spike_mask]['Days until event'].max()
                # Calculate the difference between current point and last spike
                days_since_spike = registrations_at_point['Days until event'].max() - last_spike_days
            else:
                # If no spikes, use 0 to indicate no prior spikes have occurred
                days_since_spike = 0
            #---------------------------------------------------------------------------------------------


            feature_dict = {
                'current_registrations': registrations_at_point['cumulative_registrations'].max(),
                'avg_daily_rate': registrations_at_point['registration_count'].mean(),
                'recent_velocity': recent_registrations,
                'early_velocity': early_registrations,
                'registration_acceleration': (recent_registrations - early_registrations) / 7,
                'days_active': len(registrations_at_point),
                'peak_daily_registrations': registrations_at_point['registration_count'].max(),
                'registration_volatility': registrations_at_point['registration_count'].std(),
                'spike_count': registrations_at_point['promotional_spike'].sum(),
                'days_since_last_spike': days_since_spike,
                'event_code': registrations_at_point['Event Code'].unique()[0],
                'target_audience': registrations_at_point['Target audience'].unique()[0],
                'registration_start_date': registrations_at_point['Created Date'].unique()[0],
                'event_date': registrations_at_point['Event date'].unique()[0]
                }


            features.append(feature_dict)

        return pd.DataFrame(features)
